<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/Proyecto_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq
import random
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional

In [3]:
# Estados
class EstadoPaciente(Enum):
    EN_COLA = 1
    EN_SERVICIO = 2
    ATENDIDO = 3

class TipoEvento(Enum):
    LLEGADA = 1
    FIN_SERVICIO = 2
    # El evento INICIO_SERVICIO es implícito (ocurre inmediatamente al liberar doctor)

@dataclass
class Paciente:
    id: int
    tiempo_llegada: float
    estado: EstadoPaciente = EstadoPaciente.EN_COLA
    tiempo_inicio_servicio: Optional[float] = None
    tiempo_salida: Optional[float] = None

@dataclass
class Doctor:
    id: int
    ocupado: bool = False
    paciente_actual: Optional[Paciente] = None

@dataclass(order=True)
class Evento:
    tiempo: float
    tipo: TipoEvento = field(compare=False)
    paciente: Optional[Paciente] = field(compare=False, default=None)
    doctor: Optional[Doctor] = field(compare=False, default=None)

Motor de Simulación:

In [4]:
class ClinicaMMc:
    def __init__(self, env_lambda, env_mu, c_doctores, tiempo_max_simulacion):
        self.lambd = env_lambda
        self.mu = env_mu
        self.c = c_doctores
        self.tiempo_max = tiempo_max_simulacion

        self.reloj = 0.0
        self.cola_eventos = []
        self.doctores = [Doctor(i) for i in range(c_doctores)]
        self.cola_pacientes = [] # FIFO infinita

        # Métricas
        self.total_pacientes = 0
        self.tiempos_espera_cola = []   # Wq
        self.tiempos_estancia_total = [] # W

    def agendar_evento(self, evento: Evento):
        heapq.heappush(self.cola_eventos, evento)

    def correr(self):
        # Primer evento: Primera llegada
        self.agendar_evento(Evento(0.0, TipoEvento.LLEGADA))

        while self.cola_eventos and self.reloj < self.tiempo_max:
            evento_actual = heapq.heappop(self.cola_eventos)
            self.reloj = evento_actual.tiempo

            if evento_actual.tipo == TipoEvento.LLEGADA:
                self.manejar_llegada(evento_actual)
            elif evento_actual.tipo == TipoEvento.FIN_SERVICIO:
                self.manejar_fin_servicio(evento_actual)

    def manejar_llegada(self, evento):
        # 1. Crear paciente
        self.total_pacientes += 1
        nuevo_paciente = Paciente(id=self.total_pacientes, tiempo_llegada=self.reloj)

        # 2. Programar siguiente llegada (Proceso de Poisson)
        prox_llegada = self.reloj + random.expovariate(self.lambd)
        if prox_llegada < self.tiempo_max:
            self.agendar_evento(Evento(prox_llegada, TipoEvento.LLEGADA))

        # 3. Lógica M/M/c: ¿Hay servidor libre?
        doctor_libre = self.buscar_doctor_libre()
        if doctor_libre:
            self.iniciar_servicio(nuevo_paciente, doctor_libre)
        else:
            # A la cola infinita
            self.cola_pacientes.append(nuevo_paciente)

    def manejar_fin_servicio(self, evento):
        doctor = evento.doctor
        paciente = evento.paciente

        # 1. Registrar salida y liberar doctor
        paciente.estado = EstadoPaciente.ATENDIDO
        paciente.tiempo_salida = self.reloj
        doctor.ocupado = False
        doctor.paciente_actual = None

        # 2. Guardar métricas del paciente que se va
        wq = paciente.tiempo_inicio_servicio - paciente.tiempo_llegada
        w = paciente.tiempo_salida - paciente.tiempo_llegada
        self.tiempos_espera_cola.append(wq)
        self.tiempos_estancia_total.append(w)

        # 3. Si hay cola, atender al siguiente INMEDIATAMENTE
        if self.cola_pacientes:
            siguiente = self.cola_pacientes.pop(0)
            self.iniciar_servicio(siguiente, doctor)

    def iniciar_servicio(self, paciente, doctor):
        paciente.estado = EstadoPaciente.EN_SERVICIO
        paciente.tiempo_inicio_servicio = self.reloj
        doctor.ocupado = True
        doctor.paciente_actual = paciente

        # Generar tiempo de servicio (Exponencial)
        duracion = random.expovariate(self.mu)
        self.agendar_evento(Evento(self.reloj + duracion, TipoEvento.FIN_SERVICIO, paciente, doctor))

    def buscar_doctor_libre(self):
        for doc in self.doctores:
            if not doc.ocupado:
                return doc
        return None